In [64]:
import rdflib
def create_map(file_path, file_num):
    g = rdflib.Graph()
    g.parse(file_path, 'rdf')
#     # grab language
#     lang_pred = rdflib.URIRef('http://purl.org/dc/terms/language')
#     print(lang_pred)
#     lang_obj = [i for i in g.objects(None, lang_pred)] 
#     print(lang_obj)
#     lang = lang_obj[0] if len(lang_obj) > 0 else None
#     print(lang)
#     if (not lang or str(lang) is not 'en'):
#         return None, None
    # estimate publish date from author's birth/death
    ## grab birth date
    birth_pred = rdflib.URIRef('http://www.gutenberg.org/2009/pgterms/birthdate')
    birth_obj = [i for i in g.objects(None, birth_pred)] 
    birth = birth_obj[0] if len(birth_obj) > 0 else None
    #@ grab death date
    death_pred = rdflib.URIRef('http://www.gutenberg.org/2009/pgterms/deathdate')
    death_obj = [i for i in g.objects(None, death_pred)]
    death = death_obj[0] if (len(death_obj) > 0 and birth is not None) else None
    estimated_publish = None
    ## estimate publish date
    if death and birth: 
        estimated_publish = str((int(death) + int(birth)) / 2)
    # grab uri for text
    format_pred = rdflib.URIRef('http://purl.org/dc/terms/hasFormat')
    subj = rdflib.URIRef('http://www.gutenberg.org/ebooks/{}'.format(file_num))
    fmt_link = g.objects(subj, format_pred)
    for i in fmt_link:
        if 'txt' in i: #grab the first text url
            return (str(i), estimated_publish)

In [65]:
create_map('/Users/ssami/Documents/Personal/cache/epub/787/pg787.rdf', 787)

('http://www.gutenberg.org/files/787/787.txt', '1875.0')

In [66]:
import random, os
path_prefix = '/Users/ssami/Documents/Personal/cache/epub'
all_texts = os.listdir(path_prefix)
random.shuffle(all_texts)
sample_input_train = all_texts[:100]
sample_input_test = all_texts[101:150]

In [67]:
# build random training data
file_map = {}
for i in sample_input_train: 
    file_path = os.path.join(path_prefix, i,'pg'+i+'.rdf')
    file_map[i] = create_map(file_path, i)

Validations to be done: 
1. Is the text in English?  
2. Is publish date present? 
3. Is publish date > 0

Data prep: 
1. Divide dates into buckets: 10, 50, 100 years
2. Download data and store in files for each bucket

In [68]:
def download_raw_text(url, output_path, max_content=50000, block_size=1024):
    import requests
    stream = requests.get(url, stream=True)
    with open(output_path, 'wb') as fh: 
        count = 0
        for block in stream.iter_content(block_size):
            fh.write(block)
            count += block_size
            if count > max_content:
                print(count)
                break

In [69]:
download_raw_text('http://www.gutenberg.org/cache/epub/2600/pg2600.txt', 'pg2600.txt')

50176
